## Authorise Requests

In [19]:
import requests
import os
from dotenv import load_dotenv
from requests.auth import HTTPBasicAuth

load_dotenv()

v2Server = os.getenv("V2_SERVER")
fhirServer = os.getenv("FHIR_SERVER")
toolsServer = os.getenv("V2_TOOLS")

tokenUrl = os.getenv("OAUTH2_TOKEN")
clientId = os.getenv("CLIENT_ID")
clientSecret = os.getenv("CLIENT_SECRET")

the_data = {"grant_type": "client_credentials", "scope": "system/*.*"}
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
response = requests.post(tokenUrl, auth=HTTPBasicAuth(clientId, clientSecret),verify=False,data=the_data, headers=headers)
responseInclude = response.json()
token = responseInclude['access_token']

print(token)
print("token was displayed")

/Users/kevinmayfield/github/MFT/Testing/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.20'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## HL7 v2 Orders O21

1. Convert to FHIR and save a copy
2. Convert to V2 and save a copy
3. Send to RIE FHIR Server

In [20]:

import time

order_list = ['OML_O21_RPY.txt','OML_O21_R0A.txt']
report_list = ['ORU_R01_DLIMS.txt','ORU_R01_R125.1_R0A.txt','ORU_R01_R125.1_RBS.txt','ORU_R01_R125.1_REP.txt','ORU_R01_R125.1_RR8.txt','ORU_R01_R125.1_RX1.txt','ORU_R01_R125.1_SG9.txt','ORU_R01_R125.1_ZT001.txt','ORU_R01_R125.1_7A3.txt','ORU_R01_R125.1_RPY.txt','ORU_R01_R125.1_RXK.txt']

headersFHIR = {'Content-Type': 'application/fhir+json',
               'Authorization': 'Bearer ' + token}
headersV2 = {'Content-Type': 'x-application/hl7-v2+er7'}

for file in order_list:
    with open('Input/V2/O21/' + file, 'rb') as f:
        s = f.read()

        rFHIR = requests.post(toolsServer + '/transformToFHIR', data = s,verify=False,headers=headersV2)
        ##print(r.status_code)
        with open('Output/FHIR/O21/' + file + '.json', 'w') as vFHIR:
            vFHIR.write(rFHIR.text)
            vFHIR.close()

        rV2 = requests.post(toolsServer + '/transformToV2', data = rFHIR.text,verify=False,headers=headersFHIR)
        ##print(r.status_code)
        with open('Output/V2/O21/' + file, 'w') as v2:
            v2.write(rV2.text)
            v2.close()

        response = requests.post(fhirServer + '$process-message', verify=False,data=rFHIR.text, headers=headersFHIR)
        ##print(response.status_code)
        if response.status_code != 200:
            print(response.text)
            time.sleep(4)
            response = requests.post(fhirServer + '$process-message', verify=False,data=rFHIR.text, headers=headersFHIR)
            print(response.status_code)
            if response.status_code != 200:
                print("Retry Failed")


/Users/kevinmayfield/github/MFT/Testing/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.20'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## HL7 v2 Reports R01

1. Convert from v2 to FHIR,
2. Save FHIR Output
3. post to server
4. Convert back to V2 and save

In [21]:

import json

for file in report_list:
    with open('Input/V2/R01/' + file, 'rb') as g:
        s = g.read()

        rFHIR = requests.post(toolsServer + '/transformToFHIR', data = s,verify=False,headers=headersV2)
        ##print(r.status_code)
        with open('Output/FHIR/R01/' + file + '.json', 'w') as vFHIR:
            vFHIR.write(rFHIR.text)
            vFHIR.close()

        rV2 = requests.post(toolsServer + '/transformToV2', data = rFHIR.text,verify=False,headers=headersFHIR)
        ##print(r.status_code)
        with open('Output/V2/R01/' + file, 'w') as v2:
            v2.write(rV2.text)
            v2.close()

        r = requests.post(v2Server, data = s)
        print(r.status_code)
        print(r.text)

        response1JSON = rFHIR.json()
        for entry in response1JSON['entry']:
            resource = entry['resource']
            if resource['resourceType'] == 'MessageHeader':
                resource['eventCoding']['code'] = 'T02'
                entry['resource'] = resource
        jsonStr = json.dumps(response1JSON, indent=4)
        with open('Output/FHIR/T02/MDM_T02_' + file + '.json', 'w') as vFHIRT02:
            vFHIRT02.write(jsonStr)
            vFHIRT02.close()
        rV2T02 = requests.post(toolsServer + '/transformToV2', data = jsonStr,verify=False,headers=headersFHIR)
        ##print(r.status_code)
        with open('Output/V2/T02/MDM_T02_' + file, 'w') as v2:
            v2.write(rV2T02.text)
            v2.close()


200
MSA|CA|ORIE-251024-9|ISC|IGENE|MFT|202602030628||ACK^R01|ORIE-251024-9|T|2.3
